Group 6 \\
Νικολόπουλος Ευάγγελος \\
Γεώργιος Σάντης

# Task 1
##Scenario of Assignment 3 Question 1

###Code from Assignment 3 (Policy Iteration - For Comparison)

In [ ]:
import numpy as np

gamma = 0
r1H = 0.1
r1L = -0.02
r2H = 0.05
r2L = -0.01
c = 0.12
p1HH = 0.9
p1HL = 0.1
p1LL = 0.9
p1LH = 0.1
p2HH = 0.9
p2HL = 0.1
p2LL = 0.9
p2LH = 0.1
N = 2

rewards = {
    (1, 'H'): r1H,
    (1, 'L'): r1L,
    (2, 'H'): r2H,
    (2, 'L'): r2L
}

transitions = {
    (1, 'H'): {'H': p1HH, 'L': p1HL},
    (1, 'L'): {'H': p1LH, 'L': p1LL},
    (2, 'H'): {'H': p2HH, 'L': p2HL},
    (2, 'L'): {'H': p2LH, 'L': p2LL}
}

states = [(1, 'H', 'H'), (1, 'H', 'L'), (1, 'L', 'H'), (1, 'L', 'L'),
          (2, 'H', 'H'), (2, 'H', 'L'), (2, 'L', 'H'), (2, 'L', 'L')]

actions = ['keep', 'switch']

def get_state_index(state):
    return states.index(state)

def get_transition_prob_reward(state, action):
    i, stock_state1, stock_state2 = state
    next_states = []
    stock_states = [stock_state1, stock_state2]

    if action == 'keep':
        if stock_states[i - 1] == 'H':
            next_states.append(((i, stock_states[i - 1], stock_states[1 - (i - 1)]), transitions[(i, 'H')]['H'], rewards[(i, 'H')]))
            stock_states[i - 1] = 'L'
            next_states.append(((i, stock_states[i - 1], stock_states[1 - (i - 1)]), transitions[(i, 'H')]['L'], rewards[(i, 'H')]))
        else:
            next_states.append(((i, stock_states[i - 1], stock_states[1 - (i - 1)]), transitions[(i, 'L')]['H'], rewards[(i, 'L')]))
            stock_states[i - 1] = 'H'
            next_states.append(((i, stock_states[i - 1], stock_states[1 - (i - 1)]), transitions[(i, 'L')]['L'], rewards[(i, 'L')]))
    else:  # Switch to a different stock
        if stock_states[1 - (i - 1)] == 'H':
            next_states.append(((2 if i == 1 else 1, stock_states[1 - (i - 1)], stock_states[i - 1]), transitions[(2 if i == 1 else 1, 'H')]['H'], rewards[(2 if i == 1 else 1, 'H')] - c))
            stock_states[1 - (i - 1)] = 'L'
            next_states.append(((2 if i == 1 else 1, stock_states[1 - (i - 1)], stock_states[i - 1]), transitions[(2 if i == 1 else 1, 'H')]['L'], rewards[(2 if i == 1 else 1, 'H')] - c))
        else:
            next_states.append(((2 if i == 1 else 1, stock_states[1 - (i - 1)], stock_states[i - 1]), transitions[(2 if i == 1 else 1, 'L')]['H'], rewards[(2 if i == 1 else 1, 'L')] - c))
            stock_states[1 - (i - 1)] = 'H'
            next_states.append(((2 if i == 1 else 1, stock_states[1 - (i - 1)], stock_states[i - 1]), transitions[(2 if i == 1 else 1, 'L')]['L'], rewards[(2 if i == 1 else 1, 'L')] - c))

    return next_states

def policy_evaluation(policy, V, theta=1e-6):
    while True:
        delta = 0
        for state in states:
            v = V[get_state_index(state)]
            action = policy[get_state_index(state)]
            new_v = 0
            for next_state, prob, reward in get_transition_prob_reward(state, action):
                new_v += prob * (reward + gamma * V[get_state_index(next_state)])
            V[get_state_index(state)] = new_v
            delta = max(delta, abs(v - new_v))
        if delta < theta:
            break
    return V

def policy_improvement(V, policy):
    policy_stable = True
    new_policy = policy.copy()
    for state in states:
        old_action = policy[get_state_index(state)]
        action_values = np.zeros(len(actions))
        for a_idx, action in enumerate(actions):
            for next_state, prob, reward in get_transition_prob_reward(state, action):
                action_values[a_idx] += prob * (reward + gamma * V[get_state_index(next_state)])
        new_action = actions[np.argmax(action_values)]
        new_policy[get_state_index(state)] = new_action
        if new_action != old_action:
            policy_stable = False
    return new_policy, policy_stable

def policy_iteration():
    V = np.zeros(len(states))
    policy = np.random.choice(actions, len(states))
    V_History = []

    while True:
        V = policy_evaluation(policy, V)
        policy, policy_stable = policy_improvement(V, policy)
        V_History.append(np.copy(V))
        if policy_stable:
            break
    return policy, V, V_History

optimal_policy, V, V_History = policy_iteration()

# Display the optimal policy
print("Optimal Policy:")
for state in states:
    print(f"State {state}: {optimal_policy[get_state_index(state)]}")

print("State Values:")
for state in states:
    print(f"State {state}: {V[get_state_index(state)]:.4f}")


###Q-Learning Algorithm for this scenario

In [ ]:
import numpy as np
import random
import itertools

# Parameters
N = 2  # Number of stocks
gamma = 0  # Discount factor
alpha = 0.1  # Learning rate
epsilon = 0.1  # Exploration rate
num_episodes = 10000  # Number of episodes for training
transaction_cost = 0.12  # Flat transaction fee

# Reward parameters
rewards = {
    1: {'H': 0.1, 'L': -0.02},
    2: {'H': 0.05, 'L': -0.01}
}

# Transition probabilities
transition_probs = {
    1: {'H': {'H': 0.9, 'L': 0.1}, 'L': {'L': 0.9, 'H': 0.1}},
    2: {'H': {'H': 0.9, 'L': 0.1}, 'L': {'L': 0.9, 'H': 0.1}}
}

# Generate all possible states
def generate_states(N):
    states = []
    for i in range(1, N + 1):
        for stock_states in itertools.product(['H', 'L'], repeat=N):
            states.append((i, *stock_states))
    return states

states = generate_states(N)
actions = list(range(1, N + 1))

# Initialize Q-table
Q = {state: {action: 0 for action in actions} for state in states}

# Define the reward function based on the provided parameters
def get_reward(state, action):
    current_stock, *stock_states = state
    current_state = stock_states[current_stock - 1]
    reward = rewards[action][current_state]
    if action != current_stock:
        reward -= transaction_cost  # Apply transaction cost
    return reward

# Define the transition function based on the provided parameters
def get_next_state(state, action):
    current_stock, *stock_states = state
    next_stock_states = []
    for i in range(N):
        current_state = stock_states[i]
        next_state = 'H' if random.random() < transition_probs[i + 1][current_state]['H'] else 'L'
        next_stock_states.append(next_state)
    next_state = (action, *next_stock_states)
    return next_state

# Q-learning algorithm
for episode in range(num_episodes):
    # Initialize the state
    state = random.choice(states)

    for t in range(100):  # Assume a finite horizon for each episode
        if random.random() < epsilon:
            action = random.choice(actions)  # Explore
        else:
            action = max(Q[state], key=Q[state].get)  # Exploit

        # Get the reward and next state
        reward = get_reward(state, action)
        next_state = get_next_state(state, action)

        # Update Q-value
        best_next_action = max(Q[next_state], key=Q[next_state].get)
        Q[state][action] += alpha * (reward + gamma * Q[next_state][best_next_action] - Q[state][action])

        # Move to the next state
        state = next_state

# Extract the optimal policy and state values
optimal_policy = {}
state_values = {}
for state in states:
    best_action = max(Q[state], key=Q[state].get)
    if state[0] == best_action:
        optimal_policy[state] = 'keep'
    else:
        optimal_policy[state] = f'switch to stock {best_action}'
    state_values[state] = max(Q[state].values())

# Print the optimal policy and state values
print("Optimal Policy:")
for state, action in optimal_policy.items():
    print(f"State {state}: {action}")

print("State Values:")
for state, value in state_values.items():
    print(f"State {state}: {value:.4f}")


##Scenario of Assignment 3 Question 2
###Code from Assignment 3 (Policy Iteration - For Comparison)

In [ ]:
import numpy as np

gamma = 0.9
r1H = 0.1
r1L = -0.02
r2H = 0.05
r2L = -0.01
c = 0.03
p1HH = 0.9
p1HL = 0.1
p1LL = 0.9
p1LH = 0.1
p2HH = 0.9
p2HL = 0.1
p2LL = 0.9
p2LH = 0.1
N = 2

rewards = {
    (1, 'H'): r1H,
    (1, 'L'): r1L,
    (2, 'H'): r2H,
    (2, 'L'): r2L
}

transitions = {
    (1, 'H'): {'H': p1HH, 'L': p1HL},
    (1, 'L'): {'H': p1LH, 'L': p1LL},
    (2, 'H'): {'H': p2HH, 'L': p2HL},
    (2, 'L'): {'H': p2LH, 'L': p2LL}
}

states = [(1, 'H', 'H'), (1, 'H', 'L'), (1, 'L', 'H'), (1, 'L', 'L'),
          (2, 'H', 'H'), (2, 'H', 'L'), (2, 'L', 'H'), (2, 'L', 'L')]

actions = ['keep', 'switch']

def get_state_index(state):
    return states.index(state)

def get_transition_prob_reward(state, action):
    i, stock_state1, stock_state2 = state
    next_states = []
    stock_states = [stock_state1, stock_state2]

    if action == 'keep':
        if stock_states[i - 1] == 'H':
            next_states.append(((i, stock_states[i - 1], stock_states[1 - (i - 1)]), transitions[(i, 'H')]['H'], rewards[(i, 'H')]))
            stock_states[i - 1] = 'L'
            next_states.append(((i, stock_states[i - 1], stock_states[1 - (i - 1)]), transitions[(i, 'H')]['L'], rewards[(i, 'H')]))
        else:
            next_states.append(((i, stock_states[i - 1], stock_states[1 - (i - 1)]), transitions[(i, 'L')]['H'], rewards[(i, 'L')]))
            stock_states[i - 1] = 'H'
            next_states.append(((i, stock_states[i - 1], stock_states[1 - (i - 1)]), transitions[(i, 'L')]['L'], rewards[(i, 'L')]))
    else:  # Switch to a different stock
        if stock_states[1 - (i - 1)] == 'H':
            next_states.append(((2 if i == 1 else 1, stock_states[1 - (i - 1)], stock_states[i - 1]), transitions[(2 if i == 1 else 1, 'H')]['H'], rewards[(2 if i == 1 else 1, 'H')] - c))
            stock_states[1 - (i - 1)] = 'L'
            next_states.append(((2 if i == 1 else 1, stock_states[1 - (i - 1)], stock_states[i - 1]), transitions[(2 if i == 1 else 1, 'H')]['L'], rewards[(2 if i == 1 else 1, 'H')] - c))
        else:
            next_states.append(((2 if i == 1 else 1, stock_states[1 - (i - 1)], stock_states[i - 1]), transitions[(2 if i == 1 else 1, 'L')]['H'], rewards[(2 if i == 1 else 1, 'L')] - c))
            stock_states[1 - (i - 1)] = 'H'
            next_states.append(((2 if i == 1 else 1, stock_states[1 - (i - 1)], stock_states[i - 1]), transitions[(2 if i == 1 else 1, 'L')]['L'], rewards[(2 if i == 1 else 1, 'L')] - c))

    return next_states

def policy_evaluation(policy, V, theta=1e-6):
    while True:
        delta = 0
        for state in states:
            v = V[get_state_index(state)]
            action = policy[get_state_index(state)]
            new_v = 0
            for next_state, prob, reward in get_transition_prob_reward(state, action):
                new_v += prob * (reward + gamma * V[get_state_index(next_state)])
            V[get_state_index(state)] = new_v
            delta = max(delta, abs(v - new_v))
        if delta < theta:
            break
    return V

def policy_improvement(V, policy):
    policy_stable = True
    new_policy = policy.copy()
    for state in states:
        old_action = policy[get_state_index(state)]
        action_values = np.zeros(len(actions))
        for a_idx, action in enumerate(actions):
            for next_state, prob, reward in get_transition_prob_reward(state, action):
                action_values[a_idx] += prob * (reward + gamma * V[get_state_index(next_state)])
        new_action = actions[np.argmax(action_values)]
        new_policy[get_state_index(state)] = new_action
        if new_action != old_action:
            policy_stable = False
    return new_policy, policy_stable

def policy_iteration():
    V = np.zeros(len(states))
    policy = np.random.choice(actions, len(states))
    V_History = []

    while True:
        V = policy_evaluation(policy, V)
        policy, policy_stable = policy_improvement(V, policy)
        V_History.append(np.copy(V))
        if policy_stable:
            break
    return policy, V, V_History

optimal_policy, V, V_History = policy_iteration()

# Display the optimal policy
print("Optimal Policy:")
for state in states:
    print(f"State {state}: {optimal_policy[get_state_index(state)]}")

print("State Values:")
for state in states:
    print(f"State {state}: {V[get_state_index(state)]:.4f}")


###Q-Learning Algorithm for this scenario

In [ ]:
import numpy as np
import random
import itertools

# Parameters
N = 2  # Number of stocks
gamma = 0.9  # Discount factor
alpha = 0.1  # Learning rate
epsilon = 1  # Exploration rate
epsilon_decay = 0.995  # Decay rate for exploration rate
num_episodes = 1000 # Number of episodes for training
transaction_cost = 0.03  # Flat transaction fee

# Reward parameters
rewards = {
    1: {'H': 0.1, 'L': -0.02},
    2: {'H': 0.05, 'L': -0.01}
}

# Transition probabilities
transition_probs = {
    1: {'H': {'H': 0.9, 'L': 0.1}, 'L': {'L': 0.9, 'H': 0.1}},
    2: {'H': {'H': 0.9, 'L': 0.1}, 'L': {'L': 0.9, 'H': 0.1}}
}

# Generate all possible states
def generate_states(N):
    states = []
    for i in range(1, N + 1):
        for stock_states in itertools.product(['H', 'L'], repeat=N):
            states.append((i, *stock_states))
    return states

states = generate_states(N)
actions = list(range(1, N + 1))

# Initialize Q-table
Q = {state: {action: 0 for action in actions} for state in states}

# Define the reward function based on the provided parameters
def get_reward(state, action):
    current_stock, *stock_states = state
    current_state = stock_states[current_stock - 1]
    reward = rewards[action][current_state]
    if action != current_stock:
        reward -= transaction_cost  # Apply transaction cost
    return reward

# Define the transition function based on the provided parameters
def get_next_state(state, action):
    current_stock, *stock_states = state
    next_stock_states = []
    for i in range(N):
        current_state = stock_states[i]
        next_state = 'H' if random.random() < transition_probs[i + 1][current_state]['H'] else 'L'
        next_stock_states.append(next_state)
    next_state = (action, *next_stock_states)
    return next_state

# Q-learning algorithm
for episode in range(num_episodes):
    # Initialize the state
    state = random.choice(states)

    for t in range(500):  # Assume a finite horizon for each episode
        if random.random() < epsilon:
            action = random.choice(actions)  # Explore
        else:
            action = max(Q[state], key=Q[state].get)  # Exploit

        # Get the reward and next state
        reward = get_reward(state, action)
        next_state = get_next_state(state, action)

        # Update Q-value
        best_next_action = max(Q[next_state], key=Q[next_state].get)
        Q[state][action] += alpha * (reward + gamma * Q[next_state][best_next_action] - Q[state][action])

        # Move to the next state
        state = next_state
    epsilon *= epsilon_decay

# Extract the optimal policy and state values
optimal_policy = {}
state_values = {}
for state in states:
    best_action = max(Q[state], key=Q[state].get)
    if state[0] == best_action:
        optimal_policy[state] = 'keep'
    else:
        optimal_policy[state] = f'switch to stock {best_action}'
    state_values[state] = max(Q[state].values())

# Print the optimal policy and state values
print("Optimal Policy:")
for state, action in optimal_policy.items():
    print(f"State {state}: {action}")

print("State Values:")
for state, value in state_values.items():
    print(f"State {state}: {value:.4f}")


#Task 2 (8 Stocks)
##Scenario of Assignment 3 Question 2
###Code from Assignment 3 (Policy Iteration - For Comparison)
**IMPORTANT: EACH TIME IT RUNS IT GENERATES THE TRANSITION PROBABILITIES AND REWARDS OF THE STOCKS CSV FILE, THAT IS USED FOR ENVIRONMENT INITIALIZATION IN TASK 2 (Q-LEARNING) AND TASK 3 (DQN). CHECK THE REPORT FOR MORE INFO**

In [ ]:
import numpy as np
import csv
import itertools

# Environment parameters
N = 8  # Number of stocks, you can change this to any value
c = 0.1  # Transaction fee
gamma = 0.99  # Discount factor

# Reward and transition probabilities
#np.random.seed(0)  # For reproducibility
r_H = np.random.uniform(-0.02, 0.1, N)
r_L = np.random.uniform(-0.02, 0.1, N)
p_HL = np.array([0.1 if i < N // 2 else 0.5 for i in range(N)])
p_LH = np.array([0.1 if i < N // 2 else 0.5 for i in range(N)])
p_HH = 1 - p_HL
p_LL = 1 - p_LH

# State space
states = list(itertools.product(range(N), itertools.product(['H', 'L'], repeat=N)))

# Action space
actions = list(range(N))

# Helper function to get the state index
def get_state_index(state):
    return states.index(state)

# Transition probabilities and rewards
def get_transition_prob_reward(state, action):
    i, stock_states = state
    next_states = []
    stock_states = list(stock_states)
    if action == i:  # Stay with the current stock
        if stock_states[action] == 'H':
            next_states.append(((i, tuple(stock_states)), p_HH[action], r_H[action]))
            stock_states[action] = 'L'
            next_states.append(((i, tuple(stock_states)), p_HL[action], r_H[action]))
        else:
            next_states.append(((i, tuple(stock_states)), p_LH[action], r_L[action]))
            stock_states[action] = 'H'
            next_states.append(((i, tuple(stock_states)), p_LL[action], r_L[action]))
    else:  # Switch to a different stock
        if stock_states[action] == 'H':
            next_states.append(((action, tuple(stock_states)), p_HH[action], r_H[action] - c))
            stock_states[action] = 'L'
            next_states.append(((action, tuple(stock_states)), p_HL[action], r_H[action] - c))
        else:
            next_states.append(((action, tuple(stock_states)), p_LH[action], r_L[action] - c))
            stock_states[action] = 'H'
            next_states.append(((action, tuple(stock_states)), p_LL[action], r_L[action] - c))
    return next_states

# Policy Iteration
def policy_evaluation(policy, V, theta=1e-6):
    while True:
        delta = 0
        for state in states:
            v = V[get_state_index(state)]
            action = policy[get_state_index(state)]
            new_v = 0
            for next_state, prob, reward in get_transition_prob_reward(state, action):
                new_v += prob * (reward + gamma * V[get_state_index(next_state)])
            V[get_state_index(state)] = new_v
            delta = max(delta, abs(v - new_v))
        if delta < theta:
            break
    return V

def policy_improvement(V, policy):
    policy_stable = True
    new_policy = policy.copy()
    for state in states:
        old_action = policy[get_state_index(state)]
        action_values = np.zeros(len(actions))
        for action in actions:
            for next_state, prob, reward in get_transition_prob_reward(state, action):
                action_values[action] += prob * (reward + gamma * V[get_state_index(next_state)])
        new_action = np.argmax(action_values)
        new_policy[get_state_index(state)] = new_action
        if new_action != old_action:
            policy_stable = False
    return new_policy, policy_stable

def policy_iteration():
    V = np.zeros(len(states))
    policy = np.random.choice(actions, len(states))

    while True:
        V = policy_evaluation(policy, V)
        policy, policy_stable = policy_improvement(V, policy)
        if policy_stable:
            break
    return policy, V

optimal_policy, V = policy_iteration()

# Display the optimal policy
print("Optimal Policy:")
for state in states:
    print(f"State {state}: Invest in stock {optimal_policy[get_state_index(state)]}")

print("State Values:")
for state in states:
    print(f"State {state}: {V[get_state_index(state)]:.4f}")

# Output optimal policy and state values to CSV
output_file_policy = 'optimal_policy.csv'
with open(output_file_policy, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["State", "Optimal Action", "State Value"])
    for state in states:
        writer.writerow([state, optimal_policy[get_state_index(state)], V[get_state_index(state)]])

# Output transition probabilities and rewards to CSV
output_file_params = 'stock_params.csv'
with open(output_file_params, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Stock", "p_HH", "p_LH", "p_LL", "p_HL", "r_H", "r_L"])
    for i in range(N):
        writer.writerow([i, p_HH[i], p_LH[i], p_LL[i], p_HL[i], r_H[i], r_L[i]])

print(f"Results have been written to {output_file_policy} and {output_file_params}")


###Q-Learning Algorithm for this scenario

In [ ]:
import numpy as np
import random
import itertools
import csv
import matplotlib.pyplot as plt

# Parameters
N = 8  # Adjust based on your actual problem
gamma = 0.99  # Discount factor
alpha = 0.1  # Learning rate
epsilon = 0.1  # Exploration rate
num_episodes = 15000  # Number of episodes for training

# Initialize transition probabilities and rewards from CSV file
def load_params_from_csv(file_path):
    p_HH, p_LH, p_LL, p_HL, r_H, r_L = [], [], [], [], [], []
    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            p_HH.append(float(row['p_HH']))
            p_LH.append(float(row['p_LH']))
            p_LL.append(float(row['p_LL']))
            p_HL.append(float(row['p_HL']))
            r_H.append(float(row['r_H']))
            r_L.append(float(row['r_L']))
    return np.array(p_HH), np.array(p_LH), np.array(p_LL), np.array(p_HL), np.array(r_H), np.array(r_L)

p_HH, p_LH, p_LL, p_HL, r_H, r_L = load_params_from_csv('stock_params.csv')

# Generate all possible states
def generate_states(N):
    states = []
    for i in range(1, N + 1):
        for stock_states in itertools.product(['H', 'L'], repeat=N):
            states.append((i, *stock_states))
    return states

states = generate_states(N)
actions = list(range(1, N + 1))

# Initialize Q-table
Q = {state: {action: 0 for action in actions} for state in states}

# Define the reward function using loaded parameters
def get_reward(state, action, c=0.1):
    i, *stock_states = state
    reward = r_H[action - 1] if stock_states[action - 1] == 'H' else r_L[action - 1]
    if action != i:
        reward -= c  # Apply transaction cost
    return reward

# Define the transition function using loaded parameters
def get_next_state(state, action):
    i, *stock_states = state
    next_stock_states = []
    for j, stock_state in enumerate(stock_states):
        if stock_state == 'H':
            next_stock_states.append('H' if random.random() < p_HH[j] else 'L')
        else:
            next_stock_states.append('H' if random.random() < p_LH[j] else 'L')
    next_state = (action, *next_stock_states)
    return next_state

# Lists to track metrics
average_rewards_per_episode = []
average_value_functions_per_episode = []

# Q-learning algorithm
for episode in range(num_episodes):
    # Initialize the state
    state = random.choice(states)
    total_reward = 0

    for t in range(10000):  # Assume a finite horizon for each episode
        if random.random() < epsilon:
            action = random.choice(actions)  # Explore
        else:
            action = max(Q[state], key=Q[state].get)  # Exploit

        # Get the reward and next state
        reward = get_reward(state, action)
        next_state = get_next_state(state, action)

        # Update Q-value
        best_next_action = max(Q[next_state], key=Q[next_state].get)
        Q[state][action] += alpha * (reward + gamma * Q[next_state][best_next_action] - Q[state][action])

        # Move to the next state
        state = next_state

        # Accumulate reward
        total_reward += reward

    # Compute the average reward for this episode
    average_rewards_per_episode.append(total_reward / 100)

    # Compute the average value function for this episode
    average_value_function = np.mean([max(Q[s].values()) for s in states])
    average_value_functions_per_episode.append(average_value_function)

plt.figure(figsize=(12, 5))

# Plot the average reward per episode
plt.subplot(1, 2, 1)
plt.plot(average_rewards_per_episode, label='Average Reward per Episode')
plt.xlabel('Episode')
plt.ylabel('Average Reward')
plt.title('Average Reward per Episode')

# Plot the average value function per episode
plt.subplot(1, 2, 2)
plt.plot(average_value_functions_per_episode, label='Average Value Function per Episode')
plt.xlabel('Episode')
plt.ylabel('Average Value Function')
plt.title('Average Value Function per Episode')

plt.tight_layout()
plt.show()

# Extract the optimal policy and state values
optimal_policy = {}
state_values = {}
for state in states:
    best_action = max(Q[state], key=Q[state].get)
    if state[0] == best_action:
        optimal_policy[state] = 'keep'
    else:
        optimal_policy[state] = f'switch to stock {best_action}'
    state_values[state] = max(Q[state].values())

# Print the optimal policy and state values
print("Optimal Policy:")
for state, action in optimal_policy.items():
    print(f"State {state}: {action}")

print("State Values:")
for state, value in state_values.items():
    print(f"State {state}: {value:.4f}")


#Task 3
##DQN

In [ ]:
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

# Load the stock parameters from the CSV file
file_path = 'stock_params.csv'
stock_params = pd.read_csv(file_path)

# Display the loaded stock parameters
print(stock_params)

class StockMarketEnv:
    def __init__(self, stock_params, transaction_fee, gamma):
        self.stock_params = stock_params
        self.transaction_fee = transaction_fee
        self.gamma = gamma
        self.num_stocks = stock_params.shape[0]
        self.state = None
        self.reset()

    def reset(self):
        # Initialize the state with random yields
        self.state = np.random.rand(self.num_stocks)
        return torch.tensor(self.state, dtype=torch.float32).unsqueeze(0)

    def step(self, action):
        current_stock = action
        next_state = []
        reward = 0

        for i in range(self.num_stocks):
            if random.random() < self.stock_params.iloc[i]['p_HH'] if self.state[i] > 0.5 else self.stock_params.iloc[i]['p_LL']:
                next_state.append(self.stock_params.iloc[i]['r_H'] if self.state[i] > 0.5 else self.stock_params.iloc[i]['r_L'])
            else:
                next_state.append(self.stock_params.iloc[i]['r_L'] if self.state[i] > 0.5 else self.stock_params.iloc[i]['r_H'])

        reward = next_state[current_stock]

        if action != current_stock:
            reward -= self.transaction_fee

        self.state = next_state
        done = False  # In this environment, the episode does not terminate
        return torch.tensor(next_state, dtype=torch.float32).unsqueeze(0), reward, done, {}

# Initialize the environment
env = StockMarketEnv(stock_params, transaction_fee=0.1, gamma=0.99)

class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the DQN
input_dim = env.num_stocks
output_dim = env.num_stocks
dqn = DQN(input_dim, output_dim)

def train_dqn(env, dqn, num_episodes, batch_size, gamma, epsilon_start, epsilon_end, epsilon_decay, lr):
    optimizer = optim.Adam(dqn.parameters(), lr=lr)
    memory = []
    epsilon = epsilon_start
    all_rewards = []
    all_value_functions = []

    for episode in range(num_episodes):
        state = env.reset()
        episode_rewards = 0

        for t in range(100):  # Limit the number of steps per episode
            if random.random() < epsilon:
                action = random.choice(range(env.num_stocks))
            else:
                with torch.no_grad():
                    action = dqn(state).argmax().item()

            next_state, reward, done, _ = env.step(action)
            episode_rewards += reward

            memory.append((state, action, reward, next_state, done))

            if len(memory) > batch_size:
                minibatch = random.sample(memory, batch_size)
                states, actions, rewards, next_states, dones = zip(*minibatch)

                states = torch.cat(states)
                next_states = torch.cat(next_states)

                q_values = dqn(states)
                next_q_values = dqn(next_states)

                q_target = torch.tensor(rewards, dtype=torch.float32) + gamma * next_q_values.max(1)[0] * (1 - torch.tensor(dones, dtype=torch.float32))

                q_values = q_values.gather(1, torch.tensor(actions, dtype=torch.int64).unsqueeze(1)).squeeze()

                loss = F.mse_loss(q_values, q_target)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            state = next_state
            if done:
                break

        epsilon = max(epsilon_end, epsilon_decay * epsilon)
        all_rewards.append(episode_rewards)
        all_value_functions.append(dqn(state).max().item())

        if episode % 10 == 0:
            print(f'Episode {episode}, Reward: {episode_rewards}, Epsilon: {epsilon}')

    return all_rewards, all_value_functions

# Train the DQN agent
num_episodes = 1000
batch_size = 32
gamma = 0.99
epsilon_start = 1.0
epsilon_end = 0.1
epsilon_decay = 0.995
lr = 1e-3

rewards, value_functions = train_dqn(env, dqn, num_episodes, batch_size, gamma, epsilon_start, epsilon_end, epsilon_decay, lr)


plt.figure(figsize=(12, 5))

# Plot the average reward per episode
plt.subplot(1, 2, 1)
plt.plot(rewards)
plt.xlabel('Episode')
plt.ylabel('Average Reward')
plt.title('Average Reward per Episode')

# Plot the average value function per episode
plt.subplot(1, 2, 2)
plt.plot(value_functions)
plt.xlabel('Episode')
plt.ylabel('Average Value Function')
plt.title('Average Value Function per Episode')
plt.show()
